In [72]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 27
num_classes = 3

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [73]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [74]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017-2018Rabi_NDVI1.csv', delimiter=',')

In [75]:
data = my_data[1:,2:]
print(data.shape)

(301, 27)


In [76]:
print(data.max())

0.935826175


In [77]:
import numpy as np
data = np.array(data)
print(data.shape)

(301, 27)


In [78]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data1 = pandas.read_csv('./DataForBiplab/FieldDataRabi2017-2018.csv', names=colnames)

In [79]:
ctype = data1.CropType.tolist()
print(ctype)

['CropType', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat

In [80]:
p = np.array(ctype)
print(p.shape)

(302,)


In [84]:
lbl1 = ctype
label = []
for i in range(1,302):
    if(lbl1[i] == "Wheat"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "Mustard"):
        label += [2]
    
print(label)
label=np.array(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [103]:
x1=[]
x2=[]
x3=[]
l1=[]
l2=[]
l3=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data[i])
        l1.append([1,0,0])
    if(label[i]==1):
        x2.append(data[i])
        l2.append([0,1,0])
    if(label[i]==2):
        x3.append(data[i])
        l3.append([0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)

print(l2.shape)

(142, 3)


In [86]:
label = np.array(label)
print(label.shape)

(301,)


In [87]:
from sklearn.utils import shuffle
data, label = shuffle(data, label, random_state = 0)

In [88]:
# 80% train 20% test (NDVIclean)
X_train = data[0:240,:]
X_test = data[241:,:]
y_train = label[0:240]
y_test = label[241:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(240, 27)
(60, 27)
(240,)
(60,)


In [89]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(240, 3)
(60, 3)


In [90]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.56568283
Epoch: 2 cost = 0.51226985
Epoch: 3 cost = 0.46106394
Epoch: 4 cost = 0.41555891
Epoch: 5 cost = 0.37903237
Epoch: 6 cost = 0.35278523
Epoch: 7 cost = 0.33553978
Epoch: 8 cost = 0.32456026
Epoch: 9 cost = 0.31715868
Epoch: 10 cost = 0.31146182
Epoch: 11 cost = 0.30644002
Epoch: 12 cost = 0.30163579
Epoch: 13 cost = 0.29689830
Epoch: 14 cost = 0.29221863
Epoch: 15 cost = 0.28765148
Epoch: 16 cost = 0.28328221
Epoch: 17 cost = 0.27920753
Epoch: 18 cost = 0.27551173
Epoch: 19 cost = 0.27223519
Epoch: 20 cost = 0.26934841
Epoch: 21 cost = 0.26675201
Epoch: 22 cost = 0.26431002
Epoch: 23 cost = 0.26189444
Epoch: 24 cost = 0.25941151
Epoch: 25 cost = 0.25680164
Epoch: 26 cost = 0.25402563
Epoch: 27 cost = 0.25105161
Epoch: 28 cost = 0.24784905
Epoch: 29 cost = 0.24438776
Epoch: 30 cost = 0.24064239
Epoch: 31 cost = 0.23660002
Epoch: 32 cost = 0.23226966
Epoch: 33 cost = 0.22768817
Epoch: 34 cost = 0.22291347
Epoch: 35 cost = 0.21800384
Epoch: 36 cost = 0.21299899
E

Epoch: 340 cost = 0.00126719
Epoch: 341 cost = 0.00131989
Epoch: 342 cost = 0.00141764
Epoch: 343 cost = 0.00151682
Epoch: 344 cost = 0.00151520
Epoch: 345 cost = 0.00145747
Epoch: 346 cost = 0.00138019
Epoch: 347 cost = 0.00137402
Epoch: 348 cost = 0.00123902
Epoch: 349 cost = 0.00122216
Epoch: 350 cost = 0.00140672
Epoch: 351 cost = 0.00141188
Epoch: 352 cost = 0.00114318
Epoch: 353 cost = 0.00109461
Epoch: 354 cost = 0.00149817
Epoch: 355 cost = 0.00179547
Epoch: 356 cost = 0.00132483
Epoch: 357 cost = 0.00122674
Epoch: 358 cost = 0.00121536
Epoch: 359 cost = 0.00114915
Epoch: 360 cost = 0.00143740
Epoch: 361 cost = 0.00175547
Epoch: 362 cost = 0.00160639
Epoch: 363 cost = 0.00139088
Epoch: 364 cost = 0.00158769
Epoch: 365 cost = 0.00144829
Epoch: 366 cost = 0.00123595
Epoch: 367 cost = 0.00108159
Epoch: 368 cost = 0.00105214
Epoch: 369 cost = 0.00110094
Epoch: 370 cost = 0.00115790
Epoch: 371 cost = 0.00143838
Epoch: 372 cost = 0.00169902
Epoch: 373 cost = 0.00150517
Epoch: 374 cos

Epoch: 631 cost = 0.00087873
Epoch: 632 cost = 0.00075867
Epoch: 633 cost = 0.00073025
Epoch: 634 cost = 0.00064067
Epoch: 635 cost = 0.00065271
Epoch: 636 cost = 0.00084264
Epoch: 637 cost = 0.00070477
Epoch: 638 cost = 0.00061513
Epoch: 639 cost = 0.00072211
Epoch: 640 cost = 0.00083607
Epoch: 641 cost = 0.00078519
Epoch: 642 cost = 0.00056205
Epoch: 643 cost = 0.00053211
Epoch: 644 cost = 0.00066648
Epoch: 645 cost = 0.00070416
Epoch: 646 cost = 0.00074448
Epoch: 647 cost = 0.00082435
Epoch: 648 cost = 0.00062845
Epoch: 649 cost = 0.00071908
Epoch: 650 cost = 0.00076393
Epoch: 651 cost = 0.00081277
Epoch: 652 cost = 0.00071938
Epoch: 653 cost = 0.00058827
Epoch: 654 cost = 0.00056865
Epoch: 655 cost = 0.00072252
Epoch: 656 cost = 0.00071859
Epoch: 657 cost = 0.00065395
Epoch: 658 cost = 0.00080942
Epoch: 659 cost = 0.00068463
Epoch: 660 cost = 0.00060412
Epoch: 661 cost = 0.00066028
Epoch: 662 cost = 0.00065713
Epoch: 663 cost = 0.00071594
Epoch: 664 cost = 0.00077841
Epoch: 665 cos

Epoch: 932 cost = 0.00048063
Epoch: 933 cost = 0.00055547
Epoch: 934 cost = 0.00056341
Epoch: 935 cost = 0.00047799
Epoch: 936 cost = 0.00047215
Epoch: 937 cost = 0.00038204
Epoch: 938 cost = 0.00048879
Epoch: 939 cost = 0.00064335
Epoch: 940 cost = 0.00052345
Epoch: 941 cost = 0.00053599
Epoch: 942 cost = 0.00057745
Epoch: 943 cost = 0.00051293
Epoch: 944 cost = 0.00048063
Epoch: 945 cost = 0.00042358
Epoch: 946 cost = 0.00042570
Epoch: 947 cost = 0.00040854
Epoch: 948 cost = 0.00043700
Epoch: 949 cost = 0.00049650
Epoch: 950 cost = 0.00056515
Epoch: 951 cost = 0.00062803
Epoch: 952 cost = 0.00053864
Epoch: 953 cost = 0.00061093
Epoch: 954 cost = 0.00053195
Epoch: 955 cost = 0.00046966
Epoch: 956 cost = 0.00035172
Epoch: 957 cost = 0.00037972
Epoch: 958 cost = 0.00046734
Epoch: 959 cost = 0.00050946
Epoch: 960 cost = 0.00051891
Epoch: 961 cost = 0.00050149
Epoch: 962 cost = 0.00060841
Epoch: 963 cost = 0.00060673
Epoch: 964 cost = 0.00042582
Epoch: 965 cost = 0.00049006
Epoch: 966 cos

In [97]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [98]:
print(y_test.shape)

(60, 3)


In [99]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.8166667


In [104]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
print(res1)
print(res2)
print(res3)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 2 0 0 0 0 0 0 0 1 1 0 0 0 0 0 2 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1
 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2 1 0 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 2 1 1 1 0 1 1 1 1 1 1 1 1 2]
[2 2 2 2 2 1 2 1 1 1 2 2 2 2 0 2 1 1 2 1 1 1 2 1 2 1 1 1 1 2 1 2 2 1 1 2 1
 2 2]


In [105]:
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
confusion.append([x1,x2,x3])

confusion=np.array(confusion)
print(confusion)

[[109   8   3]
 [  9 123  10]
 [  1  18  20]]


In [31]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data[0:210,:]
X_test = data[210:,:]
y_train = label[0:210]
y_test = label[210:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(210, 27)
(91, 27)
(210,)
(91,)


In [32]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(210, 3)
(91, 3)


In [33]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.22720098
Epoch: 2 cost = 0.91153874
Epoch: 3 cost = 0.73134434
Epoch: 4 cost = 0.65487261
Epoch: 5 cost = 0.62152963
Epoch: 6 cost = 0.60093684
Epoch: 7 cost = 0.58307977
Epoch: 8 cost = 0.56493962
Epoch: 9 cost = 0.54559934
Epoch: 10 cost = 0.52476780
Epoch: 11 cost = 0.50236630
Epoch: 12 cost = 0.47845566
Epoch: 13 cost = 0.45330343
Epoch: 14 cost = 0.42753987
Epoch: 15 cost = 0.40233085
Epoch: 16 cost = 0.37933226
Epoch: 17 cost = 0.36012353
Epoch: 18 cost = 0.34532624
Epoch: 19 cost = 0.33430350
Epoch: 20 cost = 0.32579193
Epoch: 21 cost = 0.31866585
Epoch: 22 cost = 0.31219560
Epoch: 23 cost = 0.30596062
Epoch: 24 cost = 0.29971776
Epoch: 25 cost = 0.29331428
Epoch: 26 cost = 0.28663474
Epoch: 27 cost = 0.27956901
Epoch: 28 cost = 0.27201011
Epoch: 29 cost = 0.26391513
Epoch: 30 cost = 0.25546732
Epoch: 31 cost = 0.24722000
Epoch: 32 cost = 0.23985130
Epoch: 33 cost = 0.23363343
Epoch: 34 cost = 0.22838590
Epoch: 35 cost = 0.22380389
Epoch: 36 cost = 0.21954427
E

Epoch: 328 cost = 0.00418131
Epoch: 329 cost = 0.00423829
Epoch: 330 cost = 0.00428550
Epoch: 331 cost = 0.00404442
Epoch: 332 cost = 0.00396235
Epoch: 333 cost = 0.00418878
Epoch: 334 cost = 0.00479837
Epoch: 335 cost = 0.00426898
Epoch: 336 cost = 0.00394365
Epoch: 337 cost = 0.00400511
Epoch: 338 cost = 0.00440221
Epoch: 339 cost = 0.00413920
Epoch: 340 cost = 0.00397309
Epoch: 341 cost = 0.00419979
Epoch: 342 cost = 0.00429608
Epoch: 343 cost = 0.00418876
Epoch: 344 cost = 0.00414068
Epoch: 345 cost = 0.00414029
Epoch: 346 cost = 0.00445719
Epoch: 347 cost = 0.00449969
Epoch: 348 cost = 0.00402742
Epoch: 349 cost = 0.00388039
Epoch: 350 cost = 0.00398721
Epoch: 351 cost = 0.00421133
Epoch: 352 cost = 0.00448601
Epoch: 353 cost = 0.00441565
Epoch: 354 cost = 0.00402905
Epoch: 355 cost = 0.00402554
Epoch: 356 cost = 0.00395167
Epoch: 357 cost = 0.00412015
Epoch: 358 cost = 0.00433748
Epoch: 359 cost = 0.00407814
Epoch: 360 cost = 0.00392143
Epoch: 361 cost = 0.00413517
Epoch: 362 cos

Epoch: 639 cost = 0.00075237
Epoch: 640 cost = 0.00100977
Epoch: 641 cost = 0.00084530
Epoch: 642 cost = 0.00070643
Epoch: 643 cost = 0.00089576
Epoch: 644 cost = 0.00101050
Epoch: 645 cost = 0.00090013
Epoch: 646 cost = 0.00085066
Epoch: 647 cost = 0.00069989
Epoch: 648 cost = 0.00079745
Epoch: 649 cost = 0.00074163
Epoch: 650 cost = 0.00078814
Epoch: 651 cost = 0.00076332
Epoch: 652 cost = 0.00081948
Epoch: 653 cost = 0.00084551
Epoch: 654 cost = 0.00065180
Epoch: 655 cost = 0.00095087
Epoch: 656 cost = 0.00117356
Epoch: 657 cost = 0.00073219
Epoch: 658 cost = 0.00068000
Epoch: 659 cost = 0.00068584
Epoch: 660 cost = 0.00070407
Epoch: 661 cost = 0.00066681
Epoch: 662 cost = 0.00070528
Epoch: 663 cost = 0.00093198
Epoch: 664 cost = 0.00082409
Epoch: 665 cost = 0.00073178
Epoch: 666 cost = 0.00077726
Epoch: 667 cost = 0.00078388
Epoch: 668 cost = 0.00070417
Epoch: 669 cost = 0.00073306
Epoch: 670 cost = 0.00076777
Epoch: 671 cost = 0.00082136
Epoch: 672 cost = 0.00074575
Epoch: 673 cos

Epoch: 932 cost = 0.00047734
Epoch: 933 cost = 0.00043496
Epoch: 934 cost = 0.00064974
Epoch: 935 cost = 0.00047974
Epoch: 936 cost = 0.00042690
Epoch: 937 cost = 0.00057795
Epoch: 938 cost = 0.00054312
Epoch: 939 cost = 0.00053585
Epoch: 940 cost = 0.00072139
Epoch: 941 cost = 0.00071917
Epoch: 942 cost = 0.00055315
Epoch: 943 cost = 0.00050276
Epoch: 944 cost = 0.00040703
Epoch: 945 cost = 0.00065058
Epoch: 946 cost = 0.00058456
Epoch: 947 cost = 0.00043330
Epoch: 948 cost = 0.00041219
Epoch: 949 cost = 0.00045265
Epoch: 950 cost = 0.00054320
Epoch: 951 cost = 0.00057256
Epoch: 952 cost = 0.00066401
Epoch: 953 cost = 0.00065581
Epoch: 954 cost = 0.00057606
Epoch: 955 cost = 0.00043273
Epoch: 956 cost = 0.00046958
Epoch: 957 cost = 0.00066256
Epoch: 958 cost = 0.00051384
Epoch: 959 cost = 0.00040752
Epoch: 960 cost = 0.00045622
Epoch: 961 cost = 0.00060893
Epoch: 962 cost = 0.00066810
Epoch: 963 cost = 0.00056395
Epoch: 964 cost = 0.00052654
Epoch: 965 cost = 0.00052259
Epoch: 966 cos

In [83]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7802198


In [35]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.8021978
